In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

from simplenet import *
from utils import *

In [2]:
trainloader = cifarloader()
model = SimpleNet()

Files already downloaded and verified


In [3]:
imgs, labels = iter(trainloader).next()

In [ ]:
for i in range(1):
    for imgs, labels in trainloader:
        img_var = Variable(imgs)
        